In [2]:
import pandas as pd

filepath = "data/ARM_stock_data.csv"
data = pd.read_csv(filepath)

print(data.head())

         Date       Open       High        Low      Close  Adj Close  \
0  2023-09-14  56.099998  66.279999  55.540001  63.590000  63.590000   
1  2023-09-15  68.629997  69.000000  60.750000  60.750000  60.750000   
2  2023-09-18  57.950001  58.741001  55.020000  58.000000  58.000000   
3  2023-09-19  56.250000  56.779999  53.880001  55.169998  55.169998   
4  2023-09-20  55.169998  55.400002  51.520000  52.910000  52.910000   

      Volume  
0  130534500  
1   74568900  
2   34571900  
3   18308600  
4   16369500  


In [3]:
print(data.isnull().sum())

data = data.dropna()

print(data.info())

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       306 non-null    object 
 1   Open       306 non-null    float64
 2   High       306 non-null    float64
 3   Low        306 non-null    float64
 4   Close      306 non-null    float64
 5   Adj Close  306 non-null    float64
 6   Volume     306 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 16.9+ KB
None


In [5]:
data['Date'] = pd.to_datetime(data['Date'])

data = data.sort_values(by="Date")

data.to_csv("data/cleaned_ARM_stock_data.csv", index=False)
print("Cleaned data saved!")

Cleaned data saved!


In [6]:
prices = data['Close'].values
print(prices[:10])

[63.59000015 60.75       58.         55.16999817 52.90999985 52.15999985
 51.31999969 54.43999863 53.52000046 52.99000168]
